# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [19]:
import numpy as np  
import pandas as pd
import scipy.stats as st

c:\Users\karol\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
pokemon = pd.read_csv(r'..\pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [4]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [15]:
pokemon["Legendary"].value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [18]:
mean_leg = pokemon["Total"].loc[pokemon["Legendary"]==True].mean()
std_leg = pokemon["Total"].loc[pokemon["Legendary"]==True].std()
print(mean_leg, std_leg)

mean_not_leg = pokemon["Total"].loc[pokemon["Legendary"]==False].mean()
std_not_leg = pokemon["Total"].loc[pokemon["Legendary"]==False].std()
print(mean_not_leg, std_not_leg)

637.3846153846154 60.93738905315346
417.21360544217686 106.76041745713022


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [43]:
legendary = pokemon.loc[pokemon["Legendary"]==True]
non_legendary = pokemon.loc[pokemon["Legendary"]==False]

# Set the Hypothesis
    # H0: μ total of non-legendary pokemon <= μ total of legendary pokemon
    # H1: μ total of non-legendary pokemon > μ total of legendary pokemon

# Choose significance / Confidence level
α = 0.05

stat, p_value = st.ttest_ind(non_legendary["Total"], legendary["Total"], equal_var=False, alternative="greater")

print(stat, p_value)

-25.8335743895517 1.0


What do you conclude from this test? Write your conclusions below.

In [23]:
if (stat > 0) & (p_value < α) :
    print("We reject the null hypothesis!")
else:
    print("We do not reject the null hypothesis!")

We do not reject the null hypothesis!


In [ ]:
# we can say, with a 95% certainty, that the average total of legendary pokemon
# IS  higher than the average total of non-legendary pokemon

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [46]:
pokemon["Type 1"].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [47]:
mean_water = pokemon["Total"].loc[pokemon["Type 1"]=="Water"].mean()
std_water = pokemon["Total"].loc[pokemon["Type 1"]=="Water"].std()
print(mean_water, std_water)

mean_not_water = pokemon["Total"].loc[pokemon["Type 1"]!="Water"].mean()
std_not_water = pokemon["Total"].loc[pokemon["Type 1"]!="Water"].std()
print(mean_not_water, std_not_water)

430.45535714285717 113.18826606431458
435.85901162790697 121.09168230208066


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [52]:
water = pokemon.loc[pokemon["Type 1"]=="Water"]
non_water = pokemon.loc[pokemon["Type 1"]!="Water"]

# Set the Hypothesis
    # H0: μ total of non-water pokemon <= μ total of water pokemon
    # H1: μ total of non-water pokemon > μ total of water pokemon

# Choose significance / Confidence level
α = 0.05

stat, p_value = st.ttest_ind(non_water["Total"], water["Total"], equal_var=False, alternative="greater")

print(stat, p_value)

0.4638681676327303 0.32169576929107246


Write your conclusion below.

In [49]:
if (stat > 0) & (p_value < α) :
    print("We reject the null hypothesis!")
else:
    print("We do not reject the null hypothesis!")

We do not reject the null hypothesis!


In [ ]:
# we can say, with a 95% certainty, that the average total of water pokemon 
# IS higher than the average total of non-water pokemon

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [56]:
# Set the Hypothesis
    # H0: μ defense score = μ attack score
    # H1: μ defense score != μ attack score

# Choose significance / Confidence level
α = 0.05

# Compute Statistic & # 5. Get p-value
stat, p_value = st.ttest_rel(pokemon["Defense"],pokemon["Attack"])
print(stat, p_value)

-4.325566393330478 1.7140303479358558e-05


Describe the results of the test in the cell below.

In [58]:
# Decide
if p_value < α :
    print("We reject the null hypothesis!")
else:
    print("We do not reject the null hypothesis!")

We reject the null hypothesis!


In [59]:
# we can say, with a 95% certainty, that the average defense score is different
# from the average attack score

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [60]:
# Set the Hypothesis
    # H0: μ special defense score = μ special attack score
    # H1: μ special defense score != μ special attack score

# Choose significance / Confidence level
α = 0.05

# Compute Statistic & # 5. Get p-value
stat, p_value = st.ttest_rel(pokemon["Sp. Def"],pokemon["Sp. Atk"])
print(stat, p_value)

-0.853986188453353 0.3933685997548122


Describe the results of the test in the cell below.

In [61]:
# Decide
if p_value < α :
    print("We reject the null hypothesis!")
else:
    print("We do not reject the null hypothesis!")

We do not reject the null hypothesis!


In [ ]:
# we can say, with a 95% certainty, that the average special defense score is different
# from the average special attack score

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [63]:
# Set the Hypothesis
    # H0: μ special defense score - μ special attack score = 0
    # H1: μ special defense score - μ special attack score != 0

# Choose significance / Confidence level
α = 0.05

# Compute Statistic & # 5. Get p-value
stat, p_value = st.ttest_1samp(pokemon["Sp. Def"] - pokemon["Sp. Atk"], 0)
print(stat, p_value)

-0.853986188453353 0.3933685997548122


In [65]:
# Decide
if p_value < α :
    print("We reject the null hypothesis!")
else:
    print("We do not reject the null hypothesis!")

We do not reject the null hypothesis!


In [64]:
# we can say, with a 95% certainty, that the difference between the average special defense 
# score and the average special attack score is 0

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

